# Robot arm kinematic control

This page introduce the concept of controlling the end-effector of a robot when its joint are controlled with velocity set-points.

## Importing Librairies

In [ ]:
# Loading a robotic toolbox
!git clone https://github.com/SherbyRobotics/pyro
import sys
sys.path.append('/content/pyro')
import pyro

# Generic python tools
import numpy as np
from IPython import display

# Defining the robot arm

Here we load a already defined class from the library including the dynamic equations defining the robot arm behavior.



In [ ]:
from pyro.dynamic import manipulator

# Dynamic model (inputs are motor torques)
torque_controlled_robot    = manipulator.TwoLinkManipulator()

torque_controlled_robot.l1 = 4.0 # length of first rigid link
torque_controlled_robot.l2 = 3.0 # length of second rigid link
torque_controlled_robot.l_domain = 10

# Kinematic only model (inputs are motor velocities)
speed_controlled_robot  = manipulator.SpeedControlledManipulator.from_manipulator( torque_controlled_robot )

robot = speed_controlled_robot # For this exercise, we will only use the kinematic model

## Showing the robot behavior with constant velocity inputs

Here we run a simulation of the robot arm with no controllers:

In [ ]:
# robot initial states [ joint 1 angle (rad),  joint 2 angle (rad), joint 1 velocity (rad/sec),  joint 1 velocity (rad/sec)]
robot.x0 = np.array([ 0.1, 0.1])

# robot constant inputs
robot.ubar = np.array([ 0.5, 1.0]) # Constant joint velocities

# run the simulation
robot.compute_trajectory( tf = 6 )

# Animate and display the simulation
ani  = robot.generate_simulation_html_video()
html = display.HTML( ani )
display.display(html)

# Kinematic controller

Here bellow you can write your own custom kinematic control law, i.e. computing the joint velocity based on actual joint position and time. Here the framework is already set in place for you to speed-up the implementation process. The following variable are pre-computed and available for you to use in the feedback law:

*   The joint angles $q$
*   The time $t$

Furthermore, based on a kinematic robot model the following properties are also pre-computed:
*   The actual end-effector position $r$
*   The Jacobian $J(q)$



In [ ]:
from pyro.control  import robotcontrollers

# global_index = 0
# r_d_array = [np.array([5,0]), np.array([3,0]),np.array([0,3]), np.array([5,0])] # Effector cartesian points
# delta_dist_array = []
# dist_vector_array = []
# normalized_vector_array = []
# time_array = []
# v_max = 1 # m/s
# a_max = 1 # m/s^2

# for i in range(0, len(r_d_array)-1):
#   # Compute des distances a parcourir
#   dist_vector_array.append(r_d_array[i+1] - r_d_array[i])
#   delta_dist_array.append(np.linalg.norm(dist_vector_array[i])) # vecteur unitaire vitesse
#   time_array.append((a_max * delta_dist_array[i] + v_max **2)/ (v_max*a_max))
#   normalized_vector_array.append(dist_vector_array[i]/delta_dist_array[i])

# dr_d_array = [np.array([-1,0]), np.array([3,0]),np.array([0,3]), np.array([5,0])] # Effector speed vectors, must be unit vectors
# max_trajectory_points = len(r_d_array)-1
class CustomKinematicController( robotcontrollers.EndEffectorKinematicController ) :

    global global_index


    #############################
    def c( self , y , r , t ):
        """
        Feedback static computation u = c(y,r,t)

        INPUTS
        y  : sensor signal vector     p x 1
        r  : reference signal vector  k x 1
        t  : time                     1 x 1

        OUTPUTS
        dq  : joint velocity vector   m x 1

        """

        # Feedback from sensors
        q = y                  # Joint angles

        # Pre-computed values based on the robot kinematic model
        J = self.J( q )        # Jacobian computation
        r = self.fwd_kin( q )  # End-effector postion


        vit_max = 1 # vitesse de déplacement
        acc_max = 2 # accélération du tool
        tf = vit_max/acc_max # temps d'accélération et déccélération

        goal1 = np.array([5,0])
        goal2 = np.array([3,0])
        goal3 = np.array([0,3])
        goal4 = np.array([0,5])

        d1 = np.linalg.norm(goal2-goal1)
        d2 = np.linalg.norm(goal3-goal2)
        d3 = np.linalg.norm(goal4-goal3)

        r1 = np.array(goal2-goal1)
        r2 = np.array(goal3-goal2)
        r3 = np.array(goal4-goal3)

        u1 = r1/d1
        u2 = r2/d2
        u3 = r3/d3

        t1 = ( vit_max**2 + d1*acc_max )/ ( acc_max * vit_max )
        t2 = ( vit_max**2 + d2*acc_max )/ ( acc_max * vit_max )
        t3 = ( vit_max**2 + d3*acc_max )/ ( acc_max * vit_max )

        # Déplacement 1
        if(t < t1):
          vit = 0
          t = t-0 # local time
          if(t < tf): # accel phase
            vit = acc_max * t
          elif(t > (t1-tf)): # deccel phase
            vit =  acc_max * (t1-t)
          else: # cste speed phase
            vit = vit_max

          dr_ga = vit*u1

        # Segment 2
        elif(t < (t1+t2) ):
          vit = 0
          t = t-t1 # local time
          if(t < tf):
            vit = acc_max * t
          elif(t > (t2-tf)):
            vit =  acc_max * (t2-t)
          else:
            vit = vit_max

          dr_ga = vit*u2

           # Segment 3
        elif(t < (t1+t2+t3) ):
          vit = 0
          t = t-(t1+t2) # local time
          if(t < tf):
            vit = acc_max * t
          elif(t > (t3-tf)):
            vit =  acc_max * (t3-t)
          else:
            vit = vit_max

          dr_ga = vit*u3

        else:
          dr_ga = np.zeros(2)

        # From effector speed to joint speed
        dq = np.linalg.inv( J ) @ dr_ga

        return dq


In [ ]:
ctl = CustomKinematicController( robot )

## Simulating the robot in closed-loop

Here we run a simulation of the robot with the controller.

In [ ]:
# Create the closed-loop system
robot_with_controller = ctl + robot

# Run the simulation
robot_with_controller.x0[0] = np.pi/2 - 0.643501
robot_with_controller.x0[1] = np.pi/2
robot_with_controller.compute_trajectory( tf = 15 )

In [ ]:
# Animate the simulation
ani = robot_with_controller.generate_simulation_html_video()
html = display.HTML(ani)
display.display(html)

In [ ]:
# Plot systems states
robot_with_controller.plot_trajectory('x')

In [ ]:
# Plot control inputs
robot_with_controller.plot_trajectory('u')